In [ ]:
# Importing directory
import keras 
import numpy as np

In [3]:
path=keras.utils.get_file('nietzsche.txt',origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

606208/600901 [==============================] - 2s 4us/step


In [33]:
text=open(path).read().lower()
print('corpus length:',len(text))

corpus length: 600901


In [34]:
maxlen=60
step=3
sentences=[]
next_chars=[]
for i in range(0,len(text)-maxlen,step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
print('Number of sequence:',len(sentences))

Number of sequence: 200281


In [35]:
chars=sorted(list(set(text)))
print('Unique characters:',len(chars))

Unique characters: 59


In [36]:
char_indices=dict((char,chars.index(char))for char in chars)
print('Vectorization......')
x=np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y=np.zeros((len(sentences),len(chars)),dtype=np.bool)
for i,sentence in enumerate(sentences):
    for t,char in enumerate(sentence):
        x[i,t,char_indices[char]]=1
    y[i,char_indices[next_chars[i]]]=1

Vectorization......


In [37]:
from keras import layers
from keras import models

In [38]:
model=models.Sequential()
model.add(layers.LSTM(128,input_shape=(maxlen,len(chars))))
model.add(layers.Dense(len(chars),activation='softmax'))

In [43]:
model.compile(
             loss='categorical_crossentropy',
             optimizer='adam')

In [49]:
def sample(preds,temperature=1.0):
    preds=np.array(preds).astype('float64')
    preds=np.log(preds)/temperature
    exp_preds=np.exp(preds)
    preds=exp_preds/np.sum(exp_preds)
    probas=np.random.multinomial(1,preds,1)
    return np.argmax(probas)

In [50]:
import random 
import sys

In [ ]:
for epoch in range(1,60):
    print('epoch',epoch)
    model.fit(x,y,batch_size=128,epochs=1)
    start_index=random.randint(0,len(text)-maxlen-1)
    generated_text=text[start_index:start_index+maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    for temperature in [0.1,0.2,0.5,0.8]:
        print('----temperature',temperature)
        sys.stdout.write(generated_text)
        for i in range(400):
            sampled=np.zeros((1,maxlen,len(chars)))
            for t,char in enumerate(generated_text):
                sampled[0,t,char_indices[char]]=1
            preds=model.predict(sampled,verbose=0)[0]
            next_index=sample(preds,temperature)
            next_char=chars[next_index]
            generated_text+=next_char
            generated_text=generated_text[1:]
            sys.stdout.write(next_char)
    

epoch 1
Epoch 1/1
179200/200281 [=========================>....] - ETA: 59s - loss: 1.9167 